In [20]:
from pathlib import Path
import typing
import json

In [21]:
path_to_savegame = Path('C:/Users/Victor/Documents/Paradox Interactive/Crusader Kings II/save games')
save_game_name = '771_01_23.ck2'
complete_path = path_to_savegame / save_game_name

In [26]:
class InfoRepresentation(typing.Protocol):
    
    @staticmethod
    def create(raw_into : str) -> typing.Self:
        pass
    
    def to_raw_string(self) -> str:
        pass
    
    def change_value(self , other : typing.Any) -> None:
        pass

In [27]:
class OneLineKeyValueInfo:
    def __init__(self , 
                 data_key : str ,
                 data_value , 
                 start_spaces : int ,
                 end_spaces : int):
        self.data_key = data_key
        self.data_value = data_value
        self.start_spaces= start_spaces
        self.end_spaces = end_spaces
    @staticmethod
    def create(raw_info : str) -> typing.Self:
        start_spaces = raw_info.count('\t')
        end_spaces = raw_info.c
        data_name , data_value = raw_info.removeprefix('\t').removesuffix('\n').split('=')
        return OneLineKeyValueInfo(
            data_key = data_name,
            data_value = data_value
        )
    
    def to_raw_string(self) -> str:
        return f'\t{self.data_key}={self.data_value}\n'
    
    def change_value(self , value : str) :
        self.data_value = value

class OneLineListInfo:
    def __init__(self , info_list : list[str]):
        self.info_list = info_list
    
    @staticmethod
    def create(raw_info : str) -> typing.Self:
        data = raw_info.removeprefix('\t').removesuffix('\n').split(' ')
        
        return OneLineListInfo(info_list = data)
    
    def to_raw_string(self) -> str:
        return '\t' + ' '.join(self.info_list) + '\n'
    
    def change_value(self , other_list : list[str]):
        self.info_list = other_list

class OneLineKeyListInfo:
    
    def __init__(self , data_key : str , data_list : OneLineListInfo):
        
        self.data_key = data_key
        self.data_list = data_list
    
    def create(raw_info : str) -> typing.Self:
        
        data_key , data_list_str = raw_info.removeprefix('\t').removesuffix('\n').split('=')
        data_list = OneLineListInfo.create(
                    raw_info = data_list_str.replace('{','\t').replace('}','\n')
        )
        return OneLineKeyListInfo(
            data_key = data_key ,
            data_list = data_list
        )
    
    def to_raw_string(self) -> str:
        data_list_str = self.data_list.to_raw_string().replace('\t','{').replace('\n','}')
        
        return f'\t{self.data_key}={data_list_str}\n'
class 

In [28]:
class SaveFileParser:
    
    def __init__(self , file_path : Path ):
        self.path = file_path
        self.generator = SaveFileParser.read_file_line_by_line(file_path = self.path)
    @staticmethod
    def read_file_line_by_line(file_path : Path):
        with open(file_path, 'r') as file:
            for line in file:
                yield line
    
    def one_line_data(self , line) -> bool:
        return '"' in line

In [29]:
line = '\tlastmonthincometable={0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.10000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000}\n'
line_info = OneLineKeyListInfo.create(raw_info=line)


In [30]:
game_generator = read_file_line_by_line(file_path = complete_path)

In [31]:
next(game_generator)

'CK2txt\n'

In [32]:
next(game_generator)

'\tversion="3.0.0.0"\n'

In [33]:
next(game_generator)

'\tdate="771.1.23"\n'

In [34]:
next(game_generator)

'\tplayer=\n'

In [35]:
next(game_generator)

'\t{\n'

In [36]:
next(game_generator)

'\t\tid=1068759\n'

In [37]:
next(game_generator)

'\t\ttype=66\n'

In [38]:
next(game_generator)

'\t}\n'

In [39]:
next(game_generator)

'\tplayer_realm="c_olvia"\n'